In [365]:
import h5py as hf
import pyretest as pr
import numpy as np
from scipy import interpolate
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt

import mne
from mne_bids import BIDSPath, read_raw_bids
from mne_nirs.signal_enhancement import (enhance_negative_correlation,
                                         short_channel_regression)
from mne.preprocessing.nirs import optical_density, beer_lambert_law, scalp_coupling_index, temporal_derivative_distribution_repair
from mne.io import read_raw_snirf

from itertools import compress
from mne import Epochs, events_from_annotations
import mne 
import mne.stats as ms
import statsmodels.api as sm
from statsmodels.tsa.stattools import ccf, ccovf

import scipy.io
import os


In [366]:
def normalize(raw_haemo,epochs):
    hold_data_all = epochs["1.0"].average() # take the average of breath holds for this subject
    hold_data_all = hold_data_all.get_data() # get the data

    max_during_hold = np.max(hold_data_all) # this should be a 1 x num_channels array

    z = raw_haemo.get_data() # extract (only long channel) data from oxy and deoxy-hemoglobin concentration matrix
    z = np.transpose(z) / max_during_hold # Divide all traces in each channel by the maximum change in concentration during the breath hold.

    # We are now in arbitrary units (deltaHb / deltaHb)
    raw_haemo._data= np.transpose(z) # replace the data in raw_haemo

    return raw_haemo, epochs

In [367]:
def individual_analysis(bids_path):
    # Read data with annotations in BIDS format
    raw_intensity = read_raw_bids(bids_path, verbose=False)

    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))
    raw_od.interpolate_bads()

    # Downsample and apply signal cleaning techniques
    raw_od.resample(1)
    raw_od = temporal_derivative_distribution_repair(raw_od)
    raw_od = short_channel_regression(raw_od)

    # Convert to haemoglobin and filter
    raw_haemo = beer_lambert_law(raw_od)
    
    ## FILTERS OUT HEART RATE
    raw_haemo = raw_haemo.filter(None, 0.4,
                                 h_trans_bandwidth=0.1, l_trans_bandwidth=0.01,
                                 verbose=False)
    raw_haemo.annotations.delete(raw_haemo.annotations.description == '15')
    
    # Apply further data cleaning techniques and extract epochs
    raw_haemo = enhance_negative_correlation(raw_haemo)

    # raw_haemo = get_long_channels(raw_haemo, min_dist=0.01)

    # Pick data channels that are actually informative
    roi_channels = mne.pick_channels(raw_haemo.info['ch_names'], include=['Left_PT','Right_PT'])
    raw_haemo = raw_haemo.copy().pick_channels(roi_channels)

    # Extract events but ignore those with
    events, event_dict = events_from_annotations(raw_haemo, verbose=False,
                                                 regexp='^(?![Ends]).*$')

    epochs = Epochs(raw_haemo, events, event_id=event_dict, tmin=-5, tmax=30,
                    reject=dict(hbo=100e-6), reject_by_annotation=True,
                    proj=True, baseline=(None, 0), detrend=1,
                    preload=True, verbose=False,event_repeated='merge')
    
    raw_haemo, epochs = normalize(raw_haemo,epochs)

    return raw_haemo, epochs

In [368]:
# define function to calculate euclidian distance between two points in 3D space
def euclidian_distance(x1,y1,z1,x2,y2,z2):
    return np.sqrt((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)

In [369]:
subjects = {}
sources = {f'S{i+1}': pd.DataFrame() for i in range(num_sources)}
detectors = {f'D{i+1}': pd.DataFrame() for i in range(num_dets)}
orientors = {orient: pd.DataFrame() for orient in orient_list}

subjects = {'sources': sources, 'detectors': detectors, 'orientors': orientors}

num_sources = 16
num_dets = 14
orient_list = ['Nz', 'RPA', 'LPA', 'Cz', 'Fz']

o = 0
s = 0
d = 0

for sub in range(1,11):
    

    subject_id = "%02d" % sub

    # Create the file path with a raw string
    file_path = r"C:\Users\dalto\Downloads\Lab Docs\optodes\sub-{0}\ses-01\scans\opto_MNI.mat".format(subject_id)
    file_path = os.path.normpath(file_path)
    print(f'bids_path: {file_path}')

    # Load the .mat file
    mat_file = scipy.io.loadmat(file_path)
    print(mat_file.keys())

    # Extract the 'elecpos' variable from the 'opto_MNI' key
    elecpos = np.concatenate(mat_file['opto_MNI']['elecpos'][0])
    display(elecpos[0].shape)

    # Convert the numpy array to a DataFrame
    elecpos_df = pd.DataFrame(elecpos)

    # display(elecpos_df)
    print(f'elecpos_df: {elecpos_df.shape}')

    

    print(f'range: {len(elecpos)}')

    for i in range(len(elecpos)):
        
        if i < 5:
            print(f'i (orient): {i}')
            source_key = orient_list[i]
            subjects['orientors'][source_key] = subjects['orientors'][source_key].append(elecpos_df.loc[i], ignore_index=True)
            o += 1
            # print(f'o: {o}')

        # Append each source to the corresponding source DataFrame
        if (i > 4) & (i < 21):
            print(f'i (source): {i}')
            source_key = f'S{i-4}'
            subjects['sources'][source_key] = subjects['sources'][source_key].append(elecpos_df.loc[i], ignore_index=True)
            s += 1
            # print(f's: {s}')

        if (i > 20):
            print(f'i (detector): {i}')
            detector_key = f'D{i-20}'
            subjects['detectors'][detector_key] = subjects['detectors'][detector_key].append(elecpos_df.loc[i], ignore_index=True)
            d += 1
            # print(f'd: {d}')
       
       
        # source_key = f'S{i+1}'
        # # print(f'source_key: {source_key}')
        # subjects[sub]['sources'][source_key] = pd.DataFrame()
        # subjects[sub]['sources'][source_key] = subjects[sub]['sources'][source_key].append(s_pos_df.loc[i], ignore_index=True)

print('source: \n')
display(subjects['sources']['S16'])
print('detector: \n')
display(subjects['detectors']['D14'])
print('orientor: \n')
display(subjects['orientors']['Fz'])

bids_path: C:\Users\dalto\Downloads\Lab Docs\optodes\sub-01\ses-01\scans\opto_MNI.mat
dict_keys(['__header__', '__version__', '__globals__', 'opto_MNI'])


(3,)

elecpos_df: (35, 3)
range: 35
i (orient): 0
i (orient): 1
i (orient): 2
i (orient): 3
i (orient): 4
i (source): 5
i (source): 6
i (source): 7
i (source): 8
i (source): 9
i (source): 10
i (source): 11
i (source): 12
i (source): 13
i (source): 14
i (source): 15
i (source): 16
i (source): 17
i (source): 18
i (source): 19
i (source): 20
i (detector): 21
i (detector): 22
i (detector): 23
i (detector): 24
i (detector): 25
i (detector): 26
i (detector): 27
i (detector): 28
i (detector): 29
i (detector): 30
i (detector): 31
i (detector): 32
i (detector): 33
i (detector): 34
bids_path: C:\Users\dalto\Downloads\Lab Docs\optodes\sub-02\ses-01\scans\opto_MNI.mat
dict_keys(['__header__', '__version__', '__globals__', 'opto_MNI'])


C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\7302344.py:49: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subjects['orientors'][source_key] = subjects['orientors'][source_key].append(elecpos_df.loc[i], ignore_index=True)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\7302344.py:57: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subjects['sources'][source_key] = subjects['sources'][source_key].append(elecpos_df.loc[i], ignore_index=True)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\7302344.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  subjects['detectors'][detector_key] = subjects['detectors'][detector_key].append(elecpos_df.loc[i], ignore_index=True)


(3,)

...

,0,1,2
0,50.362547,-92.310712,65.481502
1,47.997675,-92.799407,53.133097
2,47.611850,-96.853207,46.195842
3,50.485760,-97.555847,51.208219
4,49.017074,-99.508056,44.058732
5,45.679456,-90.975745,62.994827
6,48.480104,-89.495102,59.453000
7,50.533967,-90.152250,58.577761
8,50.873585,-92.341545,61.729181
9,47.397283,-88.948611,62.343838


detector: 



,0,1,2
0,67.780880,-95.680675,44.163735
1,62.156237,-95.841232,32.082413
2,64.546829,-98.104487,27.978524
3,69.157210,-96.300855,29.313873
4,64.546932,-102.561521,25.739954
5,60.100886,-95.804923,42.084760
6,63.030367,-93.469389,36.434120
7,64.062544,-95.169149,36.886662
8,66.230738,-95.822295,40.536635
9,66.731983,-90.699542,40.974224


orientor: 



,0,1,2
0,-0.494617,-9.842698,99.840478
1,0.880918,-13.536108,97.137714
2,0.121891,-22.243208,94.513375
3,6.093777,-15.124624,97.132625
4,0.870385,-21.499415,95.256371
5,0.547008,-9.397438,100.137388
6,-0.151619,-8.910201,100.290078
7,0.553961,-9.258672,100.180446
8,0.400899,-9.167000,100.243999
9,0.390229,-9.170770,100.240009


In [370]:
print(f'o: {o}')
print(f's: {s}')
print(f'd: {d}')

dfs = [subjects['sources'], subjects['detectors'], subjects['orientors']]

for df in dfs:
    for key in df.keys():
        df[key].columns = ['x', 'y', 'z']

        display(df[key].shape)

o: 50
s: 160
d: 140


(10, 3)

(10, 3)

(10, 3)

(10, 3)

...

(10, 3)

(10, 3)

(10, 3)

(10, 3)

(10, 3)

In [371]:
# create a new frame with the mean x,y,z coordinates for each source, detector, and orientor
means = pd.DataFrame()
for df in dfs:
    # display(df)
    for key in df.keys():
        # display(df[key])
        means[key] = df[key].mean()
        means[key] = means[key]
        # display(means[key])
        
display(means)

,S1,S2,S3,S4,S5,S6,S7,S8,S9,S10,...,D10,D11,D12,D13,D14,Nz,RPA,LPA,Cz,Fz
x,-62.247383,-79.745013,-59.143251,-89.532888,-99.561638,-90.348705,-77.928398,-56.683311,67.889662,84.816151,...,99.053500,99.212518,92.481016,66.803683,64.834461,-0.738986,-84.118572,86.280237,-2.874780,0.921283
y,67.586567,40.052486,57.998045,19.268056,-23.218634,-49.623332,-80.170254,-82.659230,60.586532,29.899773,...,-27.063942,-46.896918,-66.335806,-61.792238,-95.945407,86.866226,-15.401687,-19.558623,-118.831119,-12.815013
z,-7.224072,-12.244333,43.731661,24.159337,-4.712285,31.965363,0.611244,54.538886,-5.403150,-10.392147,...,35.346111,17.759099,2.559493,72.286859,35.619490,-39.648856,-52.492592,-48.699596,-12.530100,98.497248


In [372]:
def add_mean (df):
    mean_dist_array = np.array([])
    for i in range(len(df[key])):
        x1 = df[key]['x'].iloc[i]
        y1 = df[key]['y'].iloc[i]
        z1 = df[key]['z'].iloc[i]

        x2 = means[key]['x']
        y2 = means[key]['y']
        z2 = means[key]['z']

        mean_dist = euclidian_distance(x1,y1,z1,x2,y2,z2)

        mean_dist_array = np.append(mean_dist_array, mean_dist)

    df[key]['mean_dist'] = mean_dist_array


In [373]:
# calculate the euclidian distance between each source, detector, orientor, and their mean
mean_dist_array = np.empty(0)
mean_dist = 0
# run for S1 to S6
# for key in subjects['sources'][f'S{i}']:
import math
for df in dfs:
    for i in range(1,len(subjects['sources'])+1):
        if df == subjects['sources']:
            key = f'S{i}'
            add_mean(df)
        elif (df == subjects['detectors']) & (i < 15):
            key = f'D{i}'
            add_mean(df)
        elif (df == subjects['orientors']) & (i < 6):
            key = orient_list[i-1]
            add_mean(df)
        mean_dist_array = np.empty(0)

display(subjects['sources']['S1'])


,x,y,z,mean_dist
0,-58.669485,67.102365,-6.352039,3.714330
1,-63.202221,67.511196,-7.419831,0.977609
2,-64.822611,69.304134,-1.238232,6.738851
3,-63.714439,69.805873,-3.031859,4.965100
4,-58.907229,68.503653,-6.592127,3.520941
5,-59.342648,69.781670,-10.366128,4.809207
6,-62.966368,65.525874,-8.739186,2.656872
7,-66.770465,64.540121,-9.864712,6.059050
8,-61.300574,65.787329,-9.404823,2.981506
9,-62.777787,68.003457,-9.231780,2.118022


In [374]:
# add all sources to a single dataframe
sources_all = pd.DataFrame()
dets_all = pd.DataFrame()
ors_all = pd.DataFrame()

for i in range(1,17):
    key = f'S{i}'
    sources_all = sources_all.append(subjects['sources'][key], ignore_index=True)

for i in range(1,15):
    key = f'D{i}'
    dets_all = dets_all.append(subjects['detectors'][key], ignore_index=True)

for i in range(0,4):
    key = orient_list[i]
    ors_all = ors_all.append(subjects['orientors'][key], ignore_index=True)

# add a subject column to each dataframe that increases by 1 every row and resets to 1 every 16 the source, 14th detector, and 4th orientor
sources_all['subject'] = np.tile(np.arange(1,11),16)
dets_all['subject'] = np.tile(np.arange(1,11),14)
ors_all['subject'] = np.tile(np.arange(1,11),4)

# add a ID column to each dataframe that increases once per 16th souce, 14th detector, and 4th orientor
sources_all[f'Source ID'] = np.repeat(np.arange(1,17),10)
dets_all[f'Detector ID'] = np.repeat(np.arange(1,15),10)
ors_all[f'Orientor ID'] = np.repeat(np.arange(1,5),10)

# swap the value in the ID column of the ors_all for its key in the orient_list 
for i in range(0,4):
    ors_all['Orientor ID'].iloc[i*10:i*10+10] = orient_list[i]

# move the ID and subject columns to the front of each frame
for i in range(0,2):
    cols = list(sources_all.columns)
    cols = [cols[-1]] + cols[:-1]
    sources_all = sources_all[cols]

    cols = list(dets_all.columns)
    cols = [cols[-1]] + cols[:-1]
    dets_all = dets_all[cols]

    cols = list(ors_all.columns)
    cols = [cols[-1]] + cols[:-1]
    ors_all = ors_all[cols]

# set the index as the value in subject column
sources_all = sources_all.set_index('subject')
dets_all = dets_all.set_index('subject')
ors_all = ors_all.set_index('subject')

C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\2428226024.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sources_all = sources_all.append(subjects['sources'][key], ignore_index=True)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\2428226024.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dets_all = dets_all.append(subjects['detectors'][key], ignore_index=True)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\2428226024.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ors_all = ors_all.append(subjects['orientors'][key], ignore_index=True)
C:\Users\dalto\AppData\Local\Temp\ipykernel_20872\2428226024.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the 

In [385]:
# create a new frame with the mean x,y,z coordinates for each source, detector, and orientor
means_all = pd.DataFrame()
means_array = np.empty(0)
for df in dfs:
    # print(df)
    # for key in df.keys():
    #     display(df[key].shape)
    #     mean = df[key]['mean_dist'].mean()
    #     print(mean)
    #     means_array = np.append(means_array, mean)

    #     # add the mean distance to the mean frame
    # means_all[key] = means_array
    #     # display(means[key])

    
    if df == subjects['sources']:
        for i in range(1,len(subjects['sources'])+1):
            key = f'S{i}'
            mean = df[key]['mean_dist'].mean()
            means_array = np.append(means_array, mean)
        means_all[key] = means_array
        means_array = np.empty(0)

    elif (df == subjects['detectors']) & (i < 15):
        for i in range(1,len(subjects['detectors'])+1):
            key = f'D{i}'
            mean = df[key]['mean_dist'].mean()
            means_array = np.append(means_array, mean)
        means_all[key] = means_array
        means_array = np.empty(0)


    elif (df == subjects['orientors']) & (i < 6):
        for i in range(1,len(subjects['orientors'])+1):
            key = orient_list[i-1]
            mean = df[key]['mean_dist'].mean()
            means_array = np.append(means_array, mean)
        means_all[key] = means_array

    means_array = np.empty(0)
display(means_all)

,S16
0,3.854149
1,4.715332
2,3.393118
3,4.169569
4,4.661176
5,5.655449
6,6.615603
7,6.641749
8,4.759336
9,5.311358
